## <center>Парсинг сайта ficbook, фандом - Genshin Impact</center>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
#pip install selenium

In [3]:
#pip install webdriver-manager

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

Функция для страницы:

In [5]:
def ficbook_parcing(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    time.sleep(1)
    driver.get(url)
    r = driver.page_source
    time.sleep(1)
    driver.close()
    soup = bs(r, "html.parser")
    
    fanfics_name = soup.find_all('h3', class_="fanfic-inline-title")
    resname = []
    for name in fanfics_name:
        resname.append(str(name.select("a[href^='/readfic/']")))
    resname = [i.split('>')[1].split('\n')[0].replace('amp;', '') for i in resname]
        
    fanfics_pairings = soup.select('dl', class_="fanfic-inline-info")
    respairs = []
    count_fandom = 0
    count_pairing = 0
    for elem in fanfics_pairings:
        temp = str(elem)
        if 'Фэндом:' in temp:
            count_fandom += 1
        if 'Пэйринг и персонажи:' in temp: 
            if (count_fandom - count_pairing) == 1:
                respairs.append(bs(temp, "html.parser"))
                count_pairing += 1
            else:
                for j in range(0, count_fandom - count_pairing - 1):
                    respairs.append(bs('', "html.parser"))
                respairs.append(bs(temp, "html.parser"))
                count_pairing = count_fandom
    if count_pairing != count_fandom:
        for j in range(0, count_fandom - count_pairing):
            respairs.append(bs('', "html.parser"))
            
    for i in range(0, len(respairs)):
        respairs[i] = str(respairs[i].find_all('a', class_='pairing-link pairing-highlight'))
    for i in range(0, len(respairs)):
        respairs[i] = respairs[i].split('<a class="pairing-link pairing-highlight"')
        respairs[i].remove(respairs[i][0])
        for j in range(0, len(respairs[i])):
            respairs[i][j] = respairs[i][j].split('>')[1].split('<')[0]
            
    fanfics_raitings = soup.select('div[class^="badge-with-icon badge-rating"]')
    resraitings = []
    for elem in fanfics_raitings:
        resraitings.append(str(elem.find('span', class_='badge-text').text))
    
    return resname, respairs, resraitings

Прогон по страницам:

In [6]:
finalnames = []
finalpairs = []
finalraitings = []

for i in range(1, 102):
    base = "https://ficbook.net/fanfiction/games/genshin_impact?p="
    print(i)
    url = base + str(i)
    tempnames, temppairs, tempraitings = ficbook_parcing(url)
    for j in range(0, len(tempnames)):
        finalnames.append(tempnames[j])
        finalpairs.append(temppairs[j])
        finalraitings.append(tempraitings[j])

1


[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 7.58MB/s]


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


Сборка датасета:

In [7]:
for i in range(0, len(finalpairs)):
    if len(finalpairs[i]) == 0:
        finalpairs[i].append('без пейринга')
        finalnames[i] = [finalnames[i]]
        finalraitings[i] = [finalraitings[i]]
    else:
        finalnames[i] = [finalnames[i] for j in range(0, len(finalpairs[i]))]
        finalraitings[i] = [finalraitings[i] for j in range(0, len(finalpairs[i]))]
        

flatnames = [item for sublist in finalnames for item in sublist]
flatpairs = [item for sublist in finalpairs for item in sublist]
flatraitings = [item for sublist in finalraitings for item in sublist]
data = pd.DataFrame(list(zip(flatnames, flatraitings, flatpairs)), columns =['name', 'raiting', 'pairings'])
display(data.head(30))
print(data.info()) 

,name,raiting,pairings
0,Я... я типа Венти?,R,без пейринга
1,Echoes From The Abyss,R,без пейринга
2,Искатель,R,Тарталья/Итэр
3,Искатель,R,Лиза Минчи/Джинн Гуннхильдр
4,Искатель,R,Итто Аратаки/Горо
5,Искатель,R,Син Цю/Чун Юнь
6,Искатель,R,Бэй Доу/Нин Гуан
7,"Свет луны,освещал твои глаза.",PG-13,Бэй Доу/Нин Гуан
8,Пепельный реквием,NC-17,Сяо/Люмин
9,Пепельный реквием,NC-17,Дилюк Рагнвиндр/Джинн Гуннхильдр


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4498 entries, 0 to 4497
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      4498 non-null   object
 1   raiting   4498 non-null   object
 2   pairings  4498 non-null   object
dtypes: object(3)
memory usage: 105.5+ KB
None


In [8]:
data.to_excel('raw-ficbook-genshin.xlsx')

После сохранения оригинала посмотрим на пейринги:

In [9]:
print(data['pairings'].unique())

['без пейринга' 'Тарталья/Итэр' 'Лиза Минчи/Джинн Гуннхильдр'
 'Итто Аратаки/Горо' 'Син Цю/Чун Юнь' 'Бэй Доу/Нин Гуан' 'Сяо/Люмин'
 'Дилюк Рагнвиндр/Джинн Гуннхильдр' 'Тома/Аяка Камисато' 'Итэр/Скарамучча'
 'Дилюк Рагнвиндр/Кэйа Альберих' 'Кэйа Альберих/Дилюк Рагнвиндр'
 'Чун Юнь/Син Цю' 'Сяо/Итэр' 'Кадзуха Каэдэхара/Хэйдзо Сиканоин'
 'ОЖП/Дилюк Рагнвиндр' 'Кэйа Альберих/Розария' 'Чжун Ли/Нин Гуан'
 'Наруто Узумаки/Хината Хьюга' 'Саске Учиха/Сакура Харуно'
 'Душа Эи!Боруто Узумаки/Васаби Изуно' 'Тарталья/Люмин'
 'мельком! !Сяо/мельком! !Итэр' 'Альбедо/Итэр' 'Мельком!Янь Фэй/Ху Тао'
 'Кадзуха Каэдэхара/Люмин' 'Нин Гуан/Розария'
 'Кадзуха Каэдэхара/Скарамучча' 'Чжун Ли/Венти' 'Лололошка/Венти'
 'Сяо/Альбедо' 'Сайно/Нилу' 'Сайно/Тигнари' 'Люмин/Аяка Камисато'
 'Тарталья/Чжун Ли' 'Скарамучча/Кадзуха Каэдэхара' 'Ху Тао/Янь Фэй'
 'Венти/Безымянный Бард' 'Джинн Гуннхильдр/Лиза Минчи'
 'Аято Камисато/Тома' 'Альбедо/Сахароза' 'Эола Лоуренс/Эмбер'
 'Мона Мегистус/Фишль' 'Кэ Цин/Гань Юй' 'Скараму

Избавимся от уточнений, которые делают одинаковые пейринги как бы разными, и приведём ситуации `A/B` и `B/A` к одному виду, чтоб они также не считались разными:

In [15]:
def pairing_unification(pairing):
    if '/' in pairing:
        unified = '/'.join(sorted(pairing.split('/')))
        return unified
    else:
        return pairing

In [16]:
dupl_keywords = ['fem!', 'Упоминание!', 'упоминание!', 'упоминания!', 'упом!', 'Намёк!', 'мельком!',
                 'Мельком!', 'мельком! ', 'намек!', 'намёк!', 'немного!', 'безответно!', 'Упом!', 'Душа Эи',
                 'элементы!', 'фоном!', 'на фоне!', 'бывшие!', 'alt!', '!dark', 'альфа!', 'омега!', 'бета!', '!']

for keyword in dupl_keywords:
    data['pairings'] = data['pairings'].str.replace(keyword, '', regex=False)
data['pairings'] = data['pairings'].apply(pairing_unification)
data = data.drop_duplicates()
display(data.head(30))
print(data.info())

,name,raiting,pairings
0,Я... я типа Венти?,R,без пейринга
1,Echoes From The Abyss,R,без пейринга
2,Искатель,R,Итэр/Тарталья
3,Искатель,R,Джинн Гуннхильдр/Лиза Минчи
4,Искатель,R,Горо/Итто Аратаки
5,Искатель,R,Син Цю/Чун Юнь
6,Искатель,R,Бэй Доу/Нин Гуан
7,"Свет луны,освещал твои глаза.",PG-13,Бэй Доу/Нин Гуан
8,Пепельный реквием,NC-17,Люмин/Сяо
9,Пепельный реквием,NC-17,Джинн Гуннхильдр/Дилюк Рагнвиндр


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4349 entries, 0 to 4497
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      4349 non-null   object
 1   raiting   4349 non-null   object
 2   pairings  4349 non-null   object
dtypes: object(3)
memory usage: 135.9+ KB
None


In [17]:
#data.to_excel('ficbook-genshin.xlsx')